# Dynamic DAG

Bridgic can dynamic revise arrange and execute logic at runtime. Now let's understand them with a sample example.

<br>
<div style="text-align: center;">
<img src="../imgs/dynamic_dag.png" alt="Parameter Passing" width="400" height="250">
</div>
<br>

Take the worker dynamically added at runtime as an example. In the context of article writing, the common practice is to split multiple sub-topics based on user input and then generate text around each topic. During this process, we cannot know in advance how many sub-problems the original input will be split into. We can only determine at runtime how many subsequent writing tasks there will be. 

> Of course, we can specify that only a certain number of sub-topics be split out, but it cannot guarantee that it will be applicable to all user inputs. In user inputs from different fields, it might be necessary to stipulate multiple splitting quantities.

Now, we receive user input, break down subtopics, and then generate text. There are four steps:

1. Receive user input
2. Break down subtopics and add worker
3. Write around subtopics
4. Summary writing results

Before we start, let's prepare the running environment.

Use the `export` command to set up the API information for model invocation.

In [ ]:
# Setting environment variables in the terminal:
export VLLM_SERVER_API_BASE="your-api-url"
export VLLM_SERVER_API_KEY="your-api-key"
export VLLM_SERVER_MODEL_NAME="your-model-name"

Get the environment variables.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

Import the necessary packages.

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from bridgic.core.automa import GraphAutoma, worker, ArgsMappingRule
from bridgic.llms.vllm.vllm_server_llm import VllmServerLlm, Message, Role, PydanticModel

We initialize the model to facilitate our subsequent convenient invocation of it to complete tasks.

In [ ]:
llm = VllmServerLlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    timeout=20,
)

Furthermore, we define that when the model completes entity extraction and query expansion, it returns the result in a Pydantic data structure.

In [ ]:
class SubTopics(BaseModel):  # The expected format of the model output in the topic_split worker
    sub_topics: List[str] = Field(description="All sub-topics in the input.")

Next, let's complete the three steps of query expansion to achieve our goal:

In [ ]:
class ArticleWriter(GraphAutoma):
    @worker(is_start=True)
    async def pre_query(self, query: str):  # Receive the user's input and preprocess query
        return query

    @worker(dependencies=["pre_query"], is_output=True)
    async def topic_split(self, query: str):
        response: SubTopics = await llm.astructured_output(
            model=_model_name,
            constraint=PydanticModel(model=SubTopics),
            messages=[
                Message.from_text(text="split the input into multiple sub-topics", role=Role.SYSTEM),
                Message.from_text(text=query, role=Role.USER,),
            ]
        )
        print(response.model_dump_json(indent=4))
        res = []
        for i, sub_topic in enumerate(response.sub_topics):
            res.append(self.write_around_subtopic(sub_topic))
        return res

    async def write_around_subtopic(self, sub_topic: str):
        response: str = await llm.achat(
            model=_model_name,
            messages=[
                Message.from_text(text=f"write around the sub-topic: {sub_topic}", role=Role.SYSTEM),
                Message.from_text(text=sub_topic, role=Role.USER,),
            ]
        )
        text = response.message.content
        self.ferry_to('summary', text)
        return text

    @worker()
    def summary(self, text: str):
        return text